In [15]:
import cv2 #Image Processing
import numpy as np #Stores Images as Arrays
from matplotlib import pyplot as plt #Histograms
import os #File Parsing

Loading Images from the 'Images/' folder into a list

In [17]:
images = {}
for x in os.listdir("Images"):
    images[x] = cv2.imread("Images/" + x)

Generating a class for the Sliding Window which holds the requirement to be an N x N square with the way the 'topleft' and 'bottomright' variables are set. Moreover, the xlimit and ylimit variables describe the x and y boundaries of the image respectively.

The height variable is rather self explanatory. Although, the pace and prev variables consist of the stride or step of the sliding window, and the previous y position of the window respectively. The previous y value is stored in order to check whether the sliding window moved.

In [20]:
class Window:
    
    def __init__(self, image, n, p):
        
        self.xlimit = image.shape[1]+n
        self.ylimit = image.shape[0]+n
        self.topleft = (0,0)
        self.bottomright = (n,n)
        self.prev = n
        self.height = n
        self.pace = (p, p)
        
        try:
            self.channels = image.shape[2]
        except:
            self.channels = 1
            
    #------------------------------
    #Gets the current Position of the Window
            
    def Position(self):
        
        return self.topleft, self.bottomright
    
    #------------------------------
    #Returns the next values regarding the next position of the window, where the window can either 
    #drop to a new line or simply move on pixel to the right
    
    def NextPosition(self):
        
        #New Line Movement
        if(self.bottomright + self.pace)[0] >= (self.xlimit - self.height):
            return (0, self.topleft[1] + self.pace[1]), (self.height, self.bottomright[1] + self.pace[1])
        
        #Moving one pixel to the Right
        else:
            return (self.topleft[0] + self.pace[0], self.topleft[1]), (self.bottomright[0] + self.pace[0], self.bottomright[1])
        
    #------------------------------
    #Operates on the NextPosition return to return the new value after movement
        
    def NextMove(self):
        
        #Move Position of Window
        self.topleft, self.bottomright = self.NextPosition()
        return self.topleft, self.bottomright
    
    #------------------------------
    #Checks whether the current coordinates of the sliding window are in bounds of the image or not
    
    def InBounds(self, templeft = None, tempright = None):
        
        if templeft is None:
            templeft = self.topleft
        if tempright is None:
            tempright = self.bottomright 
            
        return tempright[0] <= self.xlimit and tempright[1] <= self.ylimit and templeft[0] >= 0 and templeft[1] >= 0
    
    #------------------------------
    #Checking if the y coordinate of the bottom right corner has changed or not
    
    def newY(self):
        
        if self.prev == self.bottomright[1]:
            return False
        
        else:
            
            self.prev = self.bottomright[1]
            return True
        
    #------------------------------
    #
        
    def getImgBounds(self, image):
        
        img = []
        
        for i in range(self.topleft[1], self.bottomright[1]):
            
            if i >= image.shape[0]:
                continue
                
            img.append(image[i][self.topleft[0]: self.bottomright[0]])
            
        if self.channels == 1:
            
            return np.resize(np.array(img), (self.height, self.height)) #self.height twice since square
        
        else:
            
            return np.resize(np.array(img), (self.height, self.height, self.channels))

In [30]:
#Sliding Window Operations
show = True #True signifies that the sliding window is shown through imshow

window = Window(images["dababy.jpg"], 50, 20) #Where 50 is the size of the window and 20 is the pace
topleft, bottomright = window.Position()

image = cv2.rectangle(images["dababy.jpg"].copy(), topleft, bottomright, (0, 0, 255)) #(0, 0 ,255) being the BGR values
cv2.imwrite("Output/dababy.png", image, [cv2.IMWRITE_PNG_COMPRESSION, 0]) #Saving the rectangle into a PNG file

tempright, templeft = window.NextPosition()

while window.InBounds(bottomright): #Iterates while the window is in bounds of the image
    #Every iteration the window moves a position as shown in * using the functions in Window
    image = cv2.rectangle(images["dababy.jpg"].copy(), topleft, bottomright, (0, 0, 255))
    if show:
        cv2.imshow("Window", image)
        cv2.waitKey(int(1/35*1000))
     
    topleft, bottomright = window.NextMove() #*
    tempright = window.NextPosition() #*
    
cv2.destroyAllWindows()

<img src="Output/dababy.png">

In [40]:
class Kernel:
    
    def __init__(self, kernel, weight):
        
        self.kernel = kernel
        self.weight = weight 
        
    def Filter(self, roi, axis = 0, channels = 1):
        
        output = []
        
        if axis == 2:
            
            #Loop over each channel
            for i in range(channels):
                
                #Multiplying with the RoI pixels with the kernel
                if channels == 1:
                    _filter = self.kernel * roi
                
                else:
                    _filter = self.kernel * roi[:, :, i]
                
                #Summing the values
                filsum = _filter.sum()
                
                if channels == 1:
                    _filter = self.kernel.T * roi
                
                else:
                    
                    _filter = self.kernel.T * roi[:, :, i]
                    
                filsum2 = _filter.sum()
                
                #Appending to a list 'Output'
                output.append((((filsum**2) + (filsum2 **2)) ** (1/2)) * self.weight)
                
            return np.array(output)
        
        else:
            
            if axis == 0:
                kernel = self.kernel 
            else:
                self.kernel.T
             
            #Loop over each channel
            for i in range(channels):
                
                #Multiplying with the RoI pixels with the kernel
                if channels == 1:
                    _filter = kernel * roi
                
                else:
                    _filter = kernel * roi[:, :, i]
                
                #Summing and Appending the Values 
                output.append(_filter.sum() * self.weight)
                
            return np.array(output)
        
        
    #Filters the image like an RoI by sliding a window
        
    def FilterImage(self, image, pace = 1, window = None, axis = 0):
        
        img = [] #Output
        line = [] #Line of Pixels
        
        #--------------------------
        #Validating Window
        if window is None:
            
            move = Window(image, self.kernel.shape[0], pace) #Generating a Window Object
            
        else:
            
            image = window.getImgBounds(image)
            move = Window(image, self.kernel.shape[0], pace)
            
        topleft, _ = move.NextPosition()
        
        #-------------------------
        #Operations when Filtering 
        while move.InBounds(topleft):
            
            roi = move.getImgBounds(image)
            
            if move.newY():
                img.append(line)
                line = []
                
            line.append(self.Filter(roi, axis, move.channels))
            
            #--------------------
            #Preparing the Window for the next iteration
            move.NextMove()
            topleft, _ = move.NextPosition() 
            
        return np.array(img) #Returning the Image as an array of Pixels 
        
   

The Kernels required are defined as below

In [32]:
class Sobel: #Hard Coded numpy array
    
    def __init__(self, weight):
        
        self.kernel = Kernel(np.array([[-1 , 0, 1],
                                       [-2, 0 , 2],
                                       [-1, 0 , 1]]), weight)
        
    def FilterImage(self, image, pace = 1, window = None, axis = 2):
        return self.kernel.FilterImage(image, pace, window, axis)
    
class Gaussian: #Generating a Kernel given a size
    
    def __init__(self, size, weight):
        
        gauss = size // 2
        
        x = np.arange(0, size, 1, float)
        
        y = x[:, np.newaxis]
        
        tempx = tempy = size // 2
        self.kernel = Kernel(np.exp(-4 * np.log(2) * ((x - tempx) ** 2 + (y - tempy) ** 2) / gauss ** 2), weight)
        
    def FilterImage(self, image, pace = 1, window = None, axis = 0):
        return self.kernel.FilterImage(image, pace, window, axis)
    
class Bilinear: #Hard Coded numpy array
    
    def __init__(self, weight):
        
        self.kernel = Kernel(np.array([[1, 2, 1],
                                       [2, 4, 2],
                                       [1, 2, 1]]), weight)
        
    def FilterImage(self, image, pace = 1, window = None, axis = 0):
        return self.kernel.FilterImage(image, pace, window, axis)
    
    
sobel = Sobel(1)

Filtering the RoI using the Sobel(X+Y) Kernel by iterating over every image and getting a RoI of size (300, 300)

In [34]:
for image in images:
    win = Window(images[image], 300, 10) #Setting Window of 300 size and 10 pace
    roi = window.getImgBounds(images[image]) #Setting (300,300) RoI
    
    cv2.imwrite("Output/RoI/" + image + "BeforeFilter.jpg", roi, [cv2.IMWRITE_PNG_COMPRESSION, 0])
    fil = sobel.FilterImage(image = images[image], window = win, axis = 2) #Sobel X+Y Kernel Operations
    cv2.imwrite("Output/RoI/" + image + "AfterFilter.jpg", fil, [cv2.IMWRITE_PNG_COMPRESSION, 0])

<img src="Images/flyingdutchman.jpg">
<img src="Output/RoI/flyingdutchman.jpgAfterFiler.jpg">

Creating a copy of the initial image 'dababy.jpg' and setting the image to greyscale representation. The image is processed the same as above but with a greyscale filter

In [36]:
image = images["dababy.jpg"].copy()
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
win = Window(image, 300, 1)
roi = win.getImgBounds(image)

cv2.imwrite("Output/RoI/Dababy_BeforeFilter.jpg", roi, [cv2.IMWRITE_PNG_COMPRESSION, 0])
fil = sobel.FilterImage(image = image, window = win)
cv2.imwrite("Output/RoI/Dababy_AfterFilter.jpg", fil, [cv2.IMWRITE_PNG_COMPRESSION, 0])

True

<img src = 'Output/RoI/Dababy_BeforeFilter.jpg'>
<img src = 'Output/RoI/Dababy_AfterFilter.jpg'>

Iterating through all the images and applying the Sobel Kernel Filter

In [12]:
for image in images:
    fil = sobel.FilterImage(image = images[image], axis = 2)
    cv2.imwrite("Output/RoI/Sobel2/" + image + "AfterFilter.jpg", fil, [cv2.IMWRITE_PNG_COMPRESSION, 0])

Initializing objects for both the remaining kernels, and setting the filters into a dictionary

In [13]:
bilinear = Bilinear(1/10)
gaussian = Gaussian(5, 1/6)

#The Weights were given as 1/10 and 1/6 because otherwise, the images would have resulted to be too bright 

filters = {"Sobel": sobel,
          "Bilinear": bilinear,
           "Gaussian": gaussian}

Iterating through every image and applying the filters

In [38]:
#The below takes quite a while to compile

for image in images:
    for fil in filters:
        if fil == "Sobel":
            filt = filters[fil].FilterImage(image=images[image], axis = 0)
            cv2.imwrite("Output/SobelX/" + image + "AfterFilter.jpg", filt,  [cv2.IMWRITE_PNG_COMPRESSION, 0])
            filt = filters[fil].FilterImage(image=images[image], axis = 0)
            cv2.imwrite("Output/SobelY/" + image + "AfterFilter.jpg", filt,  [cv2.IMWRITE_PNG_COMPRESSION, 0])
            
        else:
            filt = filters[fil].FilterImage(image = images[image])
            cv2.imwrite("Output/RoI/" + fil + "/" + image + "AfterFilter.jpg", filt, [cv2.IMWRITE_PNG_COMPRESSION, 0])

# Before and After Gaussian

Before
<img src = 'Images/dababy.jpg'>

After 
<img src = 'Output/RoI/Gaussian/dababyAfterFilter.jpg'>

# Before and After Bilinear 


Before
<img src = 'Images/dababy.jpg'>

After
<img src = 'Output/RoI/Bilinear/dababy.jpg'>